In [2]:
#sample_leaf_options = [5,10,50,100,200,500]
#for leaf_size in sample_leaf_options :
#    model = RandomForestClassifier(n_estimators= 20, oob_score = True, n_jobs = -1,random_state =42, min_samples_leaf = leaf_size)
#    model.fit(df, y.values.ravel())
#    print(f'Out-of-bag score estimate: {model.oob_score_:.3}')

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics

def loadData():
    df = pd.read_csv('/sgoinfre/goinfre/Perso/pdespres/ensdata/train_b0.csv')
    y = pd.read_csv('/sgoinfre/goinfre/Perso/pdespres/ensdata/y.csv', sep=';', usecols=[1])
    print("Datas loaded")
    return df, y

df, y = loadData()
print(df.shape)

(26500, 18124)


/sgoinfre/goinfre/Perso/pdespres/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/sgoinfre/goinfre/Perso/pdespres/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


Out-of-bag score estimate: 0.607
CPU times: user 39 s, sys: 12.8 s, total: 51.8 s
Wall time: 53.9 s


In [23]:
def buildforest(train, y):
    forest = RandomForestClassifier(n_estimators=1000, oob_score=True, n_jobs=-1, min_samples_leaf=50, random_state=42)
    forest.fit(train, y.values.ravel())
    print(f'Out-of-bag score estimate: {forest.oob_score_:.3}')
    return forest

%time forest = buildforest(df, y)

Out-of-bag score estimate: 0.664
CPU times: user 36min 40s, sys: 4min 5s, total: 40min 46s
Wall time: 37min 31s


In [14]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=.25)

# Determine the false positive and true positive rates
fpr, tpr, _ = roc_curve(y_test, forest.predict_proba(X_test)[:,1])

# Calculate the AUC
roc_auc = auc(fpr, tpr)
print('ROC AUC: %0.2f' % roc_auc)

ROC AUC: 0.91


In [21]:
importances = forest.feature_importances_
coltodrop= []
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::]

# Print the feature ranking
print("Feature ranking:")

df.drop(df.columns[coltodrop], axis=1, inplace=True)
for f in range(5000):
    coltodrop.append(indices[f])
    #print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]*1000))
df.drop(df.columns[coltodrop], axis=1, inplace=True)
df.shape

Feature ranking:


(26500, 12574)

In [24]:
df.to_csv('/sgoinfre/goinfre/Perso/pdespres/ensdata/train_b1.csv', index=False)